In [1]:
# =============================================================================
# MODEL PERSISTENCE & TRAINING OPTIMIZATION
# =============================================================================
# Colab notebook for teaching saving, checkpoints, early stopping, overfitting
# Uses IMDB sentiment analysis throughout
# =============================================================================

# =============================================================================
# CELL 1: SETUP & LOAD DATA
# =============================================================================
# Run this first - sets up everything for all demos

!pip install -q tensorflow

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import pickle
import os

print("✅ Libraries imported!")

# Load IMDB dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=5000)

# Use subset for faster training
x_train, y_train = x_train[:20000], y_train[:20000]
x_test, y_test = x_test[:5000], y_test[:5000]

# Save original for showing reviews
x_train_original = x_train.copy()
x_test_original = x_test.copy()

# Pad sequences to same length
x_train = keras.utils.pad_sequences(x_train, maxlen=100)
x_test = keras.utils.pad_sequences(x_test, maxlen=100)

# Word index for converting text
word_index = keras.datasets.imdb.get_word_index()
index_to_word = {idx + 3: word for word, idx in word_index.items()}
index_to_word[0] = '<PAD>'
index_to_word[1] = '<START>'
index_to_word[2] = '<UNKNOWN>'

def show_review(index, dataset='train'):
    """Show a review in readable text."""
    if dataset == 'train':
        data, labels = x_train_original, y_train
    else:
        data, labels = x_test_original, y_test
    text = ' '.join([index_to_word.get(num, '?') for num in data[index]])
    sentiment = "POSITIVE 😊" if labels[index] == 1 else "NEGATIVE 😞"
    print(f"\n📝 Review #{index} ({sentiment})")
    print("-" * 50)
    print(text[:500] + "..." if len(text) > 500 else text)

print(f"\n✅ Data loaded!")
print(f"   Training samples: {len(x_train)}")
print(f"   Test samples: {len(x_test)}")
print(f"   Sequence length: {x_train.shape[1]} words")

print("\n📚 Sample reviews:")
show_review(0)
show_review(1)

/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


✅ Libraries imported!
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

✅ Data loaded!
   Training samples: 20000
   Test samples: 5000
   Sequence length: 100 words

📚 Sample reviews:

📝 Review #0 (POSITIVE 😊)
--------------------------------------------------
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNKNOWN> is an amazing actor and now the same being director <UNKNOWN> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNKNOWN> ...

📝 Review #1 (NEGATIVE 😞)
--------------------------------------------------
<START> big hair big <UNKNOWN> bad music and a giant safety <UNKNOWN> these are the words to best describe this terrible movie i love cheesy horror movies an

In [2]:
# =============================================================================
# CELL 2: THREE FORMATS EXPLAINED
# =============================================================================
# Visual explanation of what each format contains

print("""
=============================================================================
THREE WAYS TO SAVE A MODEL
=============================================================================

1. SavedModel (folder)
   └── my_model/
       ├── saved_model.pb      ← Architecture
       ├── variables/          ← Weights
       └── assets/             ← Optimizer state

   Use for: Production, TensorFlow Serving

2. H5 (single file)
   └── my_model.h5             ← Everything in one file

   Use for: Sharing, simple projects

3. Weights only (smallest)
   └── my_weights.h5           ← Just the numbers

   Use for: Transfer learning
   ⚠️  Must rebuild architecture before loading!

=============================================================================
""")


THREE WAYS TO SAVE A MODEL

1. SavedModel (folder)
   └── my_model/
       ├── saved_model.pb      ← Architecture
       ├── variables/          ← Weights
       └── assets/             ← Optimizer state

   Use for: Production, TensorFlow Serving

2. H5 (single file)
   └── my_model.h5             ← Everything in one file

   Use for: Sharing, simple projects

3. Weights only (smallest)
   └── my_weights.h5           ← Just the numbers

   Use for: Transfer learning
   ⚠️  Must rebuild architecture before loading!




In [3]:
# =============================================================================
# CELL 3: TRAIN MODEL
# =============================================================================
# Train a sentiment model we'll use for saving demos

print("="*60)
print("TRAINING SENTIMENT MODEL")
print("="*60)

# Build model
model = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100, name='embedding'),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2, name='lstm'),
    layers.Dense(1, activation='sigmoid', name='output')
], name='sentiment_model')

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n📊 Model Architecture:")
model.summary()

print("\n🏋️ Training...")
history = model.fit(
    x_train, y_train,
    epochs=3,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\n✅ Training complete! Test accuracy: {test_acc:.2%}")

TRAINING SENTIMENT MODEL

📊 Model Architecture:


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sentiment_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


🏋️ Training...
Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6976 - loss: 0.6073 - val_accuracy: 0.7965 - val_loss: 0.4688
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8180 - loss: 0.4221 - val_accuracy: 0.8155 - val_loss: 0.4021
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8564 - loss: 0.3490 - val_accuracy: 0.8273 - val_loss: 0.3855

✅ Training complete! Test accuracy: 82.00%


In [4]:
# =============================================================================
# CELL 4: SAVE MODEL - 3 WAYS
# =============================================================================
# Demonstrate all three saving methods

print("="*60)
print("SAVING MODEL - THREE WAYS")
print("="*60)

# Create directory
os.makedirs('saved_models-1', exist_ok=True)

SAVING MODEL - THREE WAYS


In [5]:
# Method 1: Keras format (recommended - new default)
model.save('saved_models-1/sentiment_model.keras')
print("\n✅ Keras format saved to: saved_models/sentiment_model.keras")



✅ Keras format saved to: saved_models/sentiment_model.keras


In [6]:
# Method 2: H5 format (legacy but still works)
model.save('saved_models/sentiment_model.h5')
print("✅ H5 format saved to: saved_models/sentiment_model.h5")

✅ H5 format saved to: saved_models/sentiment_model.h5


In [7]:
# Method 3: Weights only
model.save_weights('saved_models/sentiment_weights.weights.h5')
print("✅ Weights saved to: saved_models/sentiment_weights.weights.h5")


✅ Weights saved to: saved_models/sentiment_weights.weights.h5


In [8]:
# Method 4: Export for TFServing/TFLite (production deployment)
model.export('saved_models/sentiment_export')
print("✅ Exported for TFServing to: saved_models/sentiment_export/")

INFO:tensorflow:Assets written to: saved_models/sentiment_export/assets


INFO:tensorflow:Assets written to: saved_models/sentiment_export/assets


Saved artifact at 'saved_models/sentiment_export'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  5171873168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5039548048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5171872784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5171875088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5171872976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5171874896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5171875664: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Exported for TFServing to: saved_models/sentiment_export/


In [9]:
# Show file sizes
print("\n📁 File sizes:")
!ls -lh saved_models/


📁 File sizes:
total 8016
drwxr-xr-x@ 6 kyle-anthonyhay  staff   192B Dec 30 11:31 sentiment_export
-rw-r--r--@ 1 kyle-anthonyhay  staff   2.0M Dec 30 11:31 sentiment_model.h5
-rw-r--r--@ 1 kyle-anthonyhay  staff   1.9M Dec 30 11:31 sentiment_weights.weights.h5


In [10]:
# =============================================================================
# CELL 5: LOAD AND PREDICT
# =============================================================================
# Load saved model and predict on NEW sentences

print("="*60)
print("LOADING MODEL & MAKING PREDICTIONS")
print("="*60)

# Load model
loaded_model = keras.models.load_model('saved_models-1/sentiment_model.keras')
print("✅ Model loaded from H5 file!")

# Function to predict on any text
def predict_sentiment(text):
    """Predict sentiment of a new review."""
    # Convert text to lowercase and split into words
    words = text.lower().split()

    # "I want you to remove this for loop and use tokenizer" .
    # Convert words to numbers using word_index
    sequence = []
    for word in words:
        idx = word_index.get(word)
        if idx is not None and idx < 5000:  # Only use top 5000 words
            sequence.append(idx + 3)  # Offset by 3 (reserved indices)
        else:
            sequence.append(2)  # Unknown word

    # Pad to length 100
    padded = keras.utils.pad_sequences([sequence], maxlen=100)

    # Predict
    prob = loaded_model.predict(padded, verbose=0)[0][0]
    sentiment = "POSITIVE 😊" if prob > 0.5 else "NEGATIVE 😞"

    return prob, sentiment

# Test reviews - try your own!
test_reviews = [
    "This movie was amazing! I loved every minute of it.",
    "Terrible film. Waste of time and money. Do not watch.",
    "It was okay. Nothing special but not bad either.",
    "Best movie I have ever seen! Absolutely brilliant!",
    "Boring and slow. I fell asleep halfway through."
]

print("\n🔮 Predictions on new reviews:\n")
for review in test_reviews:
    prob, sentiment = predict_sentiment(review)
    print(f"📝 \"{review[:50]}...\"" if len(review) > 50 else f"📝 \"{review}\"")
    print(f"   → {prob:.1%} confident → {sentiment}\n")

# Try your own!
print("="*60)
print("💡 TRY YOUR OWN:")
print("="*60)
my_review = "I really enjoyed this film, great acting and story"
prob, sentiment = predict_sentiment(my_review)
print(f"📝 \"{my_review}\"")
print(f"   → {prob:.1%} confident → {sentiment}")

LOADING MODEL & MAKING PREDICTIONS
✅ Model loaded from H5 file!

🔮 Predictions on new reviews:

📝 "This movie was amazing! I loved every minute of it..."
   → 58.9% confident → POSITIVE 😊

📝 "Terrible film. Waste of time and money. Do not wat..."
   → 5.2% confident → NEGATIVE 😞

📝 "It was okay. Nothing special but not bad either."
   → 8.0% confident → NEGATIVE 😞

📝 "Best movie I have ever seen! Absolutely brilliant!"
   → 53.1% confident → POSITIVE 😊

📝 "Boring and slow. I fell asleep halfway through."
   → 9.6% confident → NEGATIVE 😞

💡 TRY YOUR OWN:
📝 "I really enjoyed this film, great acting and story"
   → 83.1% confident → POSITIVE 😊


In [11]:
# =============================================================================
# CELL 13: OVERFIT MODEL
# =============================================================================

print("="*60)
print("OVERFITTING DEMO: BIG MODEL, NO REGULARIZATION")
print("="*60)

model_overfit = keras.Sequential([
    layers.Embedding(5000, 128, input_length=100),
    layers.LSTM(128),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model_overfit.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(f"📊 Parameters: {model_overfit.count_params():,}\n")

history_overfit = model_overfit.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2
)

train_acc = history_overfit.history['accuracy'][-1]
val_acc = history_overfit.history['val_accuracy'][-1]
gap = train_acc - val_acc

print(f"\n📊 Train: {train_acc:.1%} | Val: {val_acc:.1%} | Gap: {gap:.1%}")
if gap > 0.1:
    print("⚠️  Gap > 10% = OVERFITTING!")

OVERFITTING DEMO: BIG MODEL, NO REGULARIZATION


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: You tried to call `count_params` on layer 'sequential', but the layer isn't built. You can build it manually via: `layer.build(input_shape)`.